In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time

/var/folders/pc/skjhh69x6c365spmwqcs6p9h0000gn/T/ipykernel_47081/3133882361.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [43]:
url = "https://ucannualwage.ucop.edu/wage/"

# Specify the path to your ChromeDriver executable
# chrome_driver_path = '/path/to/chromedriver'

# Set up ChromeOptions
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--headless")  # Uncomment this line if you want to run in headless mode
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-extensions")

# Create a WebDriver instance
driver = webdriver.Chrome(options=chrome_options)

# Use Selenium to open the webpage and let JavaScript execute
driver.get(url)

select_year = Select(driver.find_element(By.ID, "year"))
# select_year.select_by_value("2010")

# Find the dropdown element by its ID
select = Select(driver.find_element(By.ID, 'location'))
# select.select_by_value("San Diego")

select2 = Select(driver.find_element(By.CLASS_NAME, "ui-pg-selbox"))
# select2.select_by_value("60")

# select_year = Select(driver.find_element(By.ID, "year"))
# select_year.select_by_value("2010")
# time.sleep(0.5)

# Wait for the button to be clickable (adjust timeout if needed)
button = driver.find_element(By.ID, "searchButton")
# button.click()

button2 = driver.find_element(By.ID, "next")

# Wait for a few seconds to ensure JavaScript has enough time to load data
time.sleep(2)


# # Get the page source after JavaScript execution
# html_content = driver.page_source


# Parse the HTML content with BeautifulSoup
# soup = BeautifulSoup(html_content, 'html.parser')

# table = soup.find('table', attrs={'id': 'list2'})
# data = []
df = pd.DataFrame()


# select_year = Select(driver.find_element(By.ID, 'year'))
# select_year.select_by_value("2010")
for year in range(2010, 2023):

    select_year.select_by_value(str(year))
    select.select_by_value("San Diego")
    select2.select_by_value("60")
    button.click()

    time.sleep(0.5)
    
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')

    
    span_element = driver.find_element(By.ID, "sp_1")
    span_text = span_element.text
    # print(f"Year: {year}, Span Text: {span_text}")
    # print(typeof(span_text))
    # print(span_text)
    value_as_integer = int(span_text)
    # print(f"Year: {year}, Page Count: {value_as_integer}")
    
    for i in range(value_as_integer):
        # Get the page source after JavaScript execution
        html_content = driver.page_source
    
        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')
    
        #Get the table elements you want
        table = soup.find('table', attrs={'id': 'list2'})
        
        if table:
            table_rows = table.find_all('tr')
            
            data = []
            for row in table.find_all('tr'):
                row_data = [cell.text.strip() for cell in row.find_all('td')]
                data.append(row_data)
        
            # Print the data list
            # print(data)
        
        # Append the data to the DataFrame
        df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)
        # time.sleep(10)
        # button2.click()
        next_button = driver.find_element(By.ID, "next")
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(0.5)

    # the_year += 1
    # if the_year != 2023: 
    #     string_current_year = str(the_year)
    #     select_year.select_by_value(string_current_year)
    #     select.select_by_value("San Diego")
    #     select2.select_by_value("60")
    #     button.click()
    
    
    

# driver.quit()


# Create a Pandas DataFrame
# df = pd.DataFrame(data)

# Display the DataFrame
# print(df)



Year: 2010, Page Count: 597
Year: 2011, Page Count: 613
Year: 2012, Page Count: 617
Year: 2013, Page Count: 627
Year: 2014, Page Count: 648
Year: 2015, Page Count: 666
Year: 2016, Page Count: 700
Year: 2017, Page Count: 730
Year: 2018, Page Count: 754
Year: 2019, Page Count: 785
Year: 2020, Page Count: 755
Year: 2021, Page Count: 799
Year: 2022, Page Count: 858


In [44]:
#Rename the columns and cut out the columns we don't want/need
df = df.rename(columns={2: 'Year', 4: 'First Name', 5: 'Last Name', 6: 'Job Title', 7: 'Income'})
df = df.iloc[:, [2, 4, 5, 6, 7]]  # Assign the result back to the DataFrame

In [45]:
df

,Year,First Name,Last Name,Job Title,Income
0,2010,***********,***********,ASSISTANT IV,"10,168.50"
1,2010,***********,***********,ASSISTANT II,"1,695.83"
2,2010,***********,***********,ASSISTANT III,"5,810.69"
3,2010,***********,***********,ASSISTANT IV,562.50
4,2010,***********,***********,ASSISTANT IV,"3,934.70"
...,...,...,...,...,...
775,2022,*****,*****,GSR-NO REM,"5,000.00"
776,2022,*****,*****,GSR-NO REM,239.00
777,2022,*****,*****,MUSEUM EDUCATOR 2,"39,615.00"
778,2022,*****,*****,BLANK AST 3,"47,480.00"


In [22]:
df.shape

(180, 5)

In [23]:
print(df.iloc[240:300])

Empty DataFrame
Columns: [Year, First Name, Last Name, Job Title, Income]
Index: []


In [15]:
import requests
import json
# import pandas as pd
import time

start = time.time()

post_url = 'https://ucannualwage.ucop.edu/wage/search.action'

# Specify the range of years you want to include
years = range(2010, 2023)

# Create an empty list to store the dataframes for each year
dfs = []

for year in years:
    form_data = {
        '_search': 'false',
        'nd': '1708407179006',
        'rows': '52000',
        'page': '1',
        'sidx': 'EAW_LST_NAM',
        'sord': 'asc',
        'year': str(year),  # Convert year to string
        'location': 'San Diego',
        'firstname': '',
        'lastname': '',
        'title': '',
        'startSal': '',
        'endSal': ''
    }

    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.5',
        'Content-Type': 'application/x-www-form-urlencoded',
        'X-Requested-With': 'XMLHttpRequest',
        'Origin': 'https://ucannualwage.ucop.edu',
        'Referer': 'https://ucannualwage.ucop.edu/wage/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin'
    }

    response = requests.post(post_url, data=form_data, headers=headers)

    json_data = json.loads(response.text.replace("'", '"'))
    rows = [row["cell"] for row in json_data['rows']]
    df = pd.DataFrame(rows)
    
    # Add a new column for the year
    df['Year'] = year
    
    dfs.append(df)

# Concatenate all dataframes into one
final_df = pd.concat(dfs, ignore_index=True)

# display(final_df.iloc[35780:35797])

# print("Time to parse and dataframeify:", time.time() - start)


In [16]:
final_df = final_df.rename(columns={1: 'Year', 3: 'First Name', 4: 'Last Name', 5: 'Job Title', 6: 'Income'})
final_df = final_df.iloc[:, [1, 3, 4, 5, 6]]  # Assign the result back to the DataFrame

In [17]:
display(final_df.iloc[35780:35797])

print("Time to parse and dataframeify:", time.time() - start)

,Year,First Name,Last Name,Job Title,Income
35780,2010,YVONNE FELA,ZUMAYA,"____ASSISTANT, HOSPITAL, III",37815.28
35781,2010,ROSARIO,ZUMAYA-MENDOZA,"ABSTRACTOR, PATIENT RECORD II",39808.45
35782,2010,CATHERINE L.,ZUMBERGE,PROGRAM REPRESENTATIVE III,45535.72
35783,2010,MARK ANDREW,ZUMBERGE,RESEARCH _____ - FISCAL YEAR,129700.00
35784,2010,JOHN DAVID,ZUMELLO,"CUSTODIAN, SR",12108.13
35785,2010,ELINA I,ZUNIGA,ASSISTANT PROFESSOR-ACAD YR,160533.28
35786,2010,MARIA DEL REFUGI,ZUNIGA,"CUSTODIAN, SR",27468.32
35787,2010,MARIA L,ZUNIGA,"CUSTODIAN, SR",27102.72
35788,2010,ROGELIO,ZUNIGA,CLERK,15307.26
35789,2010,MARIA LU,ZUNIGA DE NUNCI,ASSOCIATE ADJUNCT PROF-HCOMP,109550.06


Time to parse and dataframeify: 56.71682620048523


In [18]:
final_df

,Year,First Name,Last Name,Job Title,Income
0,2010,***********,***********,ASSISTANT IV,10168.50
1,2010,***********,***********,ASSISTANT II,1695.83
2,2010,***********,***********,ASSISTANT III,5810.69
3,2010,***********,***********,ASSISTANT IV,562.50
4,2010,***********,***********,ASSISTANT IV,3934.70
...,...,...,...,...,...
548579,2022,THOMAS,ZUST,ELECTR TCHN,68109.00
548580,2022,MARGARET,ZVIRZDIN,PAT RCDS ABSTRACTOR 3,94538.00
548581,2022,JANE,ZWERNEMAN,BLANK AST 3,90684.00
548582,2022,MEGAN,ZYNKIAN,CLIN NURSE 3,117645.00


In [20]:
final_df = final_df[~(final_df['First Name'].str.contains('\*') | final_df['Last Name'].str.contains('\*'))]

In [21]:
final_df

,Year,First Name,Last Name,Job Title,Income
12350,2010,ERNEST L III,AARON,"TECHNICIAN, DEVELOPMENT, IV",65210.90
12351,2010,LOIS M.,AARON,TRANSLATOR/INTERP FOR THE DEAF,1891.54
12352,2010,GREGORY A.,AARONS,ASSOCIATE PROF IN RES-HCOMP,170724.59
12353,2010,CHARLES O,ABAD,"TECHNICIAN,PATIENT DIALYSI, I",45102.64
12354,2010,CONCEPCION ABUTI,ABAD,"CLERK, SR/SECRETARY",29320.17
...,...,...,...,...,...
548579,2022,THOMAS,ZUST,ELECTR TCHN,68109.00
548580,2022,MARGARET,ZVIRZDIN,PAT RCDS ABSTRACTOR 3,94538.00
548581,2022,JANE,ZWERNEMAN,BLANK AST 3,90684.00
548582,2022,MEGAN,ZYNKIAN,CLIN NURSE 3,117645.00
